In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
import json
import pickle
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import pyperclip

from scipy.interpolate import interp1d

from lared_laughter.constants import annot_exp_path, dataset_path
from utils import CovfeeParser, get_hit_stats, interp_30fps

# Annotation processing

This file processes the covfee outputs into a single dataframe that aggregates the results of the human annotation experiments.
An additional pkl file is produced with the continuous results

In [19]:
# open examples
laughter_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'laughter_examples', 'examples_with_rect.csv'), index_col=0)
speech_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'speech_examples', 'examples_with_rect.csv'), index_col=0)
calibration_examples_df = pd.read_csv(os.path.join(annot_exp_path, 'calibration_examples', 'examples.csv'))

In [20]:
laughter_examples_df['onset_time']     = laughter_examples_df['ini_time'] - laughter_examples_df['_ini_time']
laughter_examples_df['offset_time']    = laughter_examples_df['_end_time'] - laughter_examples_df['ini_time']
speech_examples_df['onset_time']       = speech_examples_df['ini_time'] - speech_examples_df['_ini_time']
speech_examples_df['offset_time']      = speech_examples_df['_end_time'] - speech_examples_df['ini_time']
calibration_examples_df['onset_time']  = calibration_examples_df['ini_time'] - calibration_examples_df['_ini_time']
calibration_examples_df['offset_time'] = calibration_examples_df['_end_time'] - calibration_examples_df['ini_time']

In [21]:
laughter_examples = {row['hash']: row.to_dict() for _, row in laughter_examples_df.iterrows()}
speech_examples = {row['hash']: row.to_dict() for _, row in speech_examples_df.iterrows()}
calibration_examples = {row['hash']: row.to_dict() for _, row in calibration_examples_df.iterrows()}
print((len(laughter_examples), len(speech_examples), len(calibration_examples)))

(1684, 785, 6)


In [22]:
results_path_1 = os.path.join(annot_exp_path, 'covfee1')
results_path_2 = os.path.join(annot_exp_path, 'covfee2')

In [23]:
# open covfee results
hits = json.load(open('../laughter2.covfee.json', 'rb'))
parser = CovfeeParser(laughter_examples, speech_examples, calibration_examples, hits)

In [24]:
# test the parser with only one HIT
# hit_path = Path(os.path.join(results_path_1, '024a80de5a3a053d40e1b7f3086e70c34cd90fc2a9668ccc6c47f7293425a185'))
# list(parser.parse_hit(hit_path)['tasks'].values())[10]

In [25]:
all_results_1 = parser.parse_v1(results_path_1)
all_results_2 = parser.parse_v2(results_path_2)
all_results = {**all_results_1, **all_results_2}

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/c/Users/Jose/gdrive/data/lared_laughter/annotation_experiment_2/covfee1'

## Interpolate

In [ ]:
examples = []
annotations = {}
for res in all_results.values():
    for ex_hash, example in res['tasks'].items():
        examples.append(res['tasks'][ex_hash])

        try:
            continuous_data = res['continuous'][ex_hash]
        except KeyError:
            print(f'Error in instance {example["instance_id"]}')
            continue

        annot = interp_30fps(
            continuous_data,
            example_len=(example['_end_time']-example['_ini_time'])
        )
        annotations[(example['instance_id'], example['hash'], example['condition'])] = annot

In [ ]:
examples_df = pd.DataFrame.from_dict(examples)
calibration_examples_df = examples_df[examples_df['calibration']]
examples_df = examples_df[~examples_df['calibration']]

In [ ]:
# check for missing modalities
for hash in examples_df['hash'].unique():
    hash_examples = examples_df[examples_df['hash'] == hash]
    
    video_examples = hash_examples[hash_examples['condition'] == 'video']
    audio_examples = hash_examples[hash_examples['condition'] == 'audio']
    av_examples = hash_examples[hash_examples['condition'] == 'av']

    num_examples = min(len(video_examples), len(audio_examples), len(av_examples))
    assert num_examples >= 2, hash

# Correction of delays

Here I correct the delays using the Calibration samples

In [ ]:
def correct_cont_delay(arr, delay):
    # move the annotations forward and pad with zeroes at the end
    delay_in_samples = round(delay * 30)
    new_data = np.roll(arr, shift = -delay_in_samples)
    new_data[-delay_in_samples:] = 0
    return new_data

In [ ]:
def set_calibration_onsets(examples_df):
    calibration_examples_onsets = {
        'bb6337eea970487ce9cd4ff26ea78c7acc6d5d1a355b7aa50029a3229f115b21': 1.54,
        '68d229cf19eec82f37580265ea93892117dd5b559b04d22489da3593315f18e7': 3.10,
        '64a92aea9395ace7ac9d60eab34911e419fc66610cd76f1e29df4b4fd16f230f': 3.35,
        '85aac70ec91eb3be1b313b33e0b7828394bbe4e4edc6a956d1e7061dfc8b250e': 1.96,
        'ced6e78fe7940c10fbc9d7c385273e68459ca399ccb668c8123cf5a66fa99819': 2.61,
        'b438e94f7fdcc80ea7927e320946c7b407b7c850fe04baa60a07a5df7d92a711': 2.99
    }

    for hash in examples_df['hash'].unique():
        examples_df.loc[examples_df['hash'] == hash, 'gt_onset'] = calibration_examples_onsets[hash]

In [ ]:
def correct_delays(examples_df_2, calibration_examples_df, annotations):
    set_calibration_onsets(calibration_examples_df)

    # examples_df.insert(0, 'c_onset', -1)
    # examples_df.insert(0, 'c_offset', -1)
    examples_df = examples_df_2.copy()
    new_annotations = {}
    
    for instance_id in examples_df['instance_id'].unique():
        instance_examples = examples_df[examples_df['instance_id'] == instance_id]

        instance_calibration = calibration_examples_df[calibration_examples_df['instance_id'] == instance_id]

        onset_delay = (instance_calibration['onset'] - instance_calibration['gt_onset']).to_numpy()
        onset_delay = onset_delay[(onset_delay > 0) & (onset_delay < 1)]
        annotator_delay = onset_delay.mean()
        examples_df.loc[examples_df['instance_id'] == instance_id, 'c_onset'] = examples_df.loc[examples_df['instance_id'] == instance_id, 'onset'] - annotator_delay
        examples_df.loc[examples_df['instance_id'] == instance_id, 'c_offset'] = examples_df.loc[examples_df['instance_id'] == instance_id, 'offset'] - annotator_delay
        examples_df.loc[examples_df['instance_id'] == instance_id, 'onset_times'] = examples_df.loc[examples_df['instance_id'] == instance_id, 'onset_times'] - annotator_delay
        examples_df.loc[examples_df['instance_id'] == instance_id, 'offset_times'] = examples_df.loc[examples_df['instance_id'] == instance_id, 'offset_times'] - annotator_delay

        for id, row in instance_examples.iterrows():
            key = (row['instance_id'], row['hash'], row['condition'])
            new_annotations[key] = correct_cont_delay(annotations[key], annotator_delay)

    return examples_df, new_annotations

In [ ]:
corr_examples, corr_annotations = correct_delays(examples_df, calibration_examples_df, annotations)
assert len(corr_examples) == len(corr_annotations), f'{len(corr_examples)} != {len(corr_annotations)}'

# Normalize intensity and confidence

Here I normalize intensity and confidence per annotator and per condition

In [45]:
examples.loc[examples['intensity'].isna(), 'intensity'] = 0
examples.loc[~examples['pressed_key'], 'intensity'] = 0
for instance_id in examples['instance_id'].unique():
    for condition in examples['condition'].unique():
        hit_condition_examples = examples[
            (examples['instance_id'] == instance_id) & (examples['condition'] == condition)
        ]
        assert len(hit_condition_examples) in [28, 23, 24], len(hit_condition_examples)
        idxs = (examples['instance_id'] == instance_id) & (examples['condition'] == condition)

        for col in ['intensity', 'confidence']:
            mean = examples.loc[idxs, col].mean()
            examples.loc[idxs, 'norm_'+col] = examples.loc[idxs, col] - mean


# Store

In [17]:
corr_examples['onset_times'] = [str(e.tolist()) for e in corr_examples['onset_times']]
corr_examples['offset_times'] = [str(e.tolist()) for e in corr_examples['offset_times']]

In [19]:
out_path = os.path.join(annot_exp_path, 'processed')
corr_examples.to_csv(os.path.join(out_path, 'examples_without_calibration.csv'))
pickle.dump(corr_annotations, open(os.path.join(out_path, 'continuous_corrected.pkl'), 'wb'))

## HIT stats

In [ ]:
hits_df = pd.DataFrame([get_hit_stats(hit) for hit in all_results.values()]).sort_values(by=['hit_group', 'hit_num'])
hits_df.head()

In [ ]:
hits_df['annot_id'] = range(1, len(hits_df)+1)
hits_df['rating'] = hits_df['rating'].astype(float)
hits_df['duration'] = hits_df['duration'].astype(float)
hits_df['rating'].fillna(-1, inplace=True)
def format_rating(f):
    if f == -1: return '-'
    return str(int(f))+'/5'
    
pyperclip.copy(hits_df.to_latex(
    columns=['annot_id', 'hit_group', 'hit_num', 'pressed', 'intensities', 'confidences', 'duration', 'rating'], 
    header=['Annotator ID', 'G', 'N', '# positive', 'Intensity', 'Confidence', 'Time taken', 'Rating'],
    formatters={'rating': format_rating, 'duration': '{:.1f}'.format},
    index=False,
    label='tab:hit_details',
    caption='Details of the annotation HITs. \\textit{G} indicates the HIT group. HITs within the same group contain the same laughter/non-laughter samples. \\textit{N} indicates the HIT number within the group. HITs with the same \\textit{N} are identical, except for the (random) ordering of the samples within each condition. HITs with different \\textit{N} contain the same samples but assigned to different conditions. Each row corresponds to one annotator (HIT). \\textit{\# positive} indicates the number of times laughter was detected by this person. \\textit{Intensities} indicates the histogram of laughter intensities by each annotator (positive examples only). Each number (in order) corresponds to one step in the Likert scale (1-7).'
    ))

/tmp/ipykernel_2642/237448400.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  pyperclip.copy(hits_df.to_latex(


# HIT information

In [103]:
pd.set_option('display.max_colwidth', 150)
feedback_df = pd.DataFrame(hit_info).transpose()
feedback_df['annot_id'] = range(1, len(hits_df)+1)
feedback_df['feedback'].fillna('-', inplace=True)
feedback_df['rating'] = feedback_df['rating'].astype(float)
feedback_df['duration'] = feedback_df['duration'].astype(float)
feedback_df['rating'].fillna(-1, inplace=True)

In [12]:
len(examples), len(annotations), len(examples[examples['has_continuous'] == True])

(3954, 4242, 3954)

In [104]:
feedback_df.tail(10)

,duration,rating,feedback,annot_id
2_7e49a08cdbcac9a9ad7658116b8a160dfa526831777981919e31172426ecc669,72.150000,4.0,-,39
2_8bfd003817f3348a53d7292c84d0a155c4ec793a5c3ef2dfe2a5f6e0a272d505,41.950000,-1.0,-,40
2_96bc549cc2a7b0e3df2888482f5199982b80d32254f53df9f4684309d9181403,32.233333,5.0,The tools were working perfectly and the process was smooth.,41
2_bd45ce15933da0e06d4d544f27a95cb5de3147aaf592557feb5bae9fa0a5d583,39.400000,4.0,At the beginning was a little bit complicated.,42
2_be99f777836ba4bffac2e20cc4e75a717d766fd5d2ccbe236dc1147d579b490b,54.566667,5.0,It was interesting to see how much difficult was to recognize the laughs of people when it was only video. Im confident a higher quality on it wou...,43
2_c9c20f2eee48c8f1f8120bb25055b3bbb167d1f9284f31b9ee303f548f2da9bd,73.150000,4.0,I just found it a bit long but interesting and I did not have an issue with the tool.,44
2_da567d02513f82ff3b0fe0c5d6237e4226ffadf44f1d9897a2efea683c0479ff,35.483333,-1.0,-,45
2_dc19285db2b452c1d3a7d625dcf0f302ee9e2eb3f690bddcddaf7ae22e58335f,40.050000,5.0,It was a kinda long but not too much.,46
2_eda1bf4c0b1962f12a7a92dbb3159f85887d48334e764f225aeedee9b4b7db16,33.400000,5.0,-,47
2_ff9b4ba9e0d4e962b5a1c5f584409289edb280057543ca3c98cedf9a1d31b0eb,60.266667,5.0,"It was indeed a unique, one-of-a-kind research, although being a bit long, I felt alright generally. A ""break"" of some sort in the middle would be...",48


In [109]:
pyperclip.copy(
    feedback_df.to_latex(
    columns=['annot_id', 'duration', 'rating', 'feedback'], 
    header=['Annotator ID', 'Time taken (s)', 'Rating', 'Feedback'],
    # float_format="{:.1f}".format,
    formatters={'rating': format_rating, 'duration': '{:.1f}'.format},
    column_format='',
    index=False,
    label='tab:hit_feedback',
    caption='Feedback given by annotators after completing the experiment.'
    ))

/tmp/ipykernel_1949/3810216713.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  feedback_df.to_latex(
